In [47]:
import pandas as pd
import numpy as np


In [48]:
# read from excel

df = pd.read_excel('卡牌数据&数学模型.xlsx', sheet_name='赞助商牌')

In [49]:
df = df[df['编号'].notna()] # 过滤无效信息

# float to 3-char string, such as 1.0 -> '001'
df['编号'] = df['编号'].apply(lambda x: str(int(x)).zfill(3))

df.head()

,编号,中文名,英文名,强度需求,建筑条件,永久/持续效果,每次休息期间的收入,右上角给标记,需要条件,图片内容,效果1必要说明,效果2必要说明,打出本卡牌时的一次性效果,终局一次性效果,票盾帽,参照卡牌,评级
0,126,时间转换器,TIME-TURNER,3,NaN,NaN,NaN,研究,NaN,电影,倒转你的行动牌槽位序列。Reverse your Action cards' slot se...,NaN,强度12345标记叠序+倒转箭头,X:1票,-,非洲专家214,B
1,127,纽特·斯卡曼德的箱子,NEWT SCAMANDER'S CASE,3,NaN,NaN,NaN,研究,3研究,电影,翻转你动物园内每张神奇动物牌对应的标准饲养区至闲置的一面。For every FB Anim...,若你的动物园内至少有5种物种图标，获得1保护点数。,六角格内放神奇动物标+翻转箭头,5≠物种:1盾,-,科学博物馆+狒狒岩208 247,C
2,128,巫师联盟,WIZARDS' UNION,4,NaN,NaN,NaN,NaN,NaN,电影,合并面积总和至多为8格的标准饲养区。Merge standard enclosures wi...,若你的动物园内至少有4/6种不同形状的建筑，获得1/2保护点数。Gain 1 conserv...,2格+3格=5格，max8,4/6不同形状:1/2盾,-,科学实验室，棕色略扩，缝一样，高度略高，滴漏居中,D
3,129,国际巫师联合会,INTERNATIONAL CONFEDERATION OF WIZARDS,3,NaN,NaN,合作合作+双箭头,NaN,"1合作, 声望3",电影,从供应堆替换1个合作动物园。你可以拥有相同的合作动物园。Replace 1 partner ...,若你的动物园内至少有5种大洲图标，获得1保护点数。,合作合作+双箭头,5大洲:1盾,1声望,导游,C
4,130,火龙研究专家,DRAGONOLOGIST,6,NaN,任何人+龙:2票,NaN,龙,NaN,NaN,任何动物园每打出1个龙类图标，你获得2票。For each dragon icon that...,若你的动物园内至少有3/6个龙类图标，获得1/2保护点数。Gain 1 conservati...,龙巢,3/6龙:1/2盾,-,北极熊馆(一模一样的框框尺寸),B


In [50]:
# 标志转化函数
mapping = {
    '研究': 'OtherTag.Science',
    '大学': 'OtherTag.University',
    '山': 'OtherTag.Rock',
    '水': 'OtherTag.Water',
    '亚': 'ContinentTag.Asia',
    '欧': 'ContinentTag.Europe',
    '非': 'ContinentTag.Africa',
    '美': 'ContinentTag.Americas',
    '澳': 'ContinentTag.Australia',
    '龙': 'AnimalTag.Dragon',
    '鸟': 'AnimalTag.Bird',
    '食肉': 'AnimalTag.Predator',
    '爬': 'AnimalTag.Reptile',
    '食草': 'AnimalTag.Herbivore',
    '灵长': 'AnimalTag.Primate',
    '萌': 'AnimalTag.Pet',
    '熊': 'AnimalTag.Bear',
    '海': 'AnimalTag.SeaAnimal',
    '嵌': 'AnimalTag.Chimera',
}

def handle_nan_and_dash(func, return_value = ''):
    def wrapper(s):
        if pd.isna(s) or s == '-':
            return return_value
        return func(s)
    return wrapper

# 定义原始的 string_tag_to_ts_enum 函数
def string_tag_to_ts_enum(s) -> list:
    enum_tags = []
    for k, v in mapping.items():
        if s.find(k) != -1:
            enum_tags.append(v)
    return enum_tags

# 使用装饰器处理 NaN 或者 '-' 值
string_tag_to_ts_enum = handle_nan_and_dash(string_tag_to_ts_enum, [])

# 将处理后的结果赋值给新列 'tags'
df = df.assign(tags=df['右上角给标记'].apply(string_tag_to_ts_enum))
df = df.assign(requirements=df['需要条件'].apply(string_tag_to_ts_enum))

df.head()

,编号,中文名,英文名,强度需求,建筑条件,永久/持续效果,每次休息期间的收入,右上角给标记,需要条件,图片内容,效果1必要说明,效果2必要说明,打出本卡牌时的一次性效果,终局一次性效果,票盾帽,参照卡牌,评级,tags,requirements
0,126,时间转换器,TIME-TURNER,3,NaN,NaN,NaN,研究,NaN,电影,倒转你的行动牌槽位序列。Reverse your Action cards' slot se...,NaN,强度12345标记叠序+倒转箭头,X:1票,-,非洲专家214,B,[OtherTag.Science],[]
1,127,纽特·斯卡曼德的箱子,NEWT SCAMANDER'S CASE,3,NaN,NaN,NaN,研究,3研究,电影,翻转你动物园内每张神奇动物牌对应的标准饲养区至闲置的一面。For every FB Anim...,若你的动物园内至少有5种物种图标，获得1保护点数。,六角格内放神奇动物标+翻转箭头,5≠物种:1盾,-,科学博物馆+狒狒岩208 247,C,[OtherTag.Science],[OtherTag.Science]
2,128,巫师联盟,WIZARDS' UNION,4,NaN,NaN,NaN,NaN,NaN,电影,合并面积总和至多为8格的标准饲养区。Merge standard enclosures wi...,若你的动物园内至少有4/6种不同形状的建筑，获得1/2保护点数。Gain 1 conserv...,2格+3格=5格，max8,4/6不同形状:1/2盾,-,科学实验室，棕色略扩，缝一样，高度略高，滴漏居中,D,[],[]
3,129,国际巫师联合会,INTERNATIONAL CONFEDERATION OF WIZARDS,3,NaN,NaN,合作合作+双箭头,NaN,"1合作, 声望3",电影,从供应堆替换1个合作动物园。你可以拥有相同的合作动物园。Replace 1 partner ...,若你的动物园内至少有5种大洲图标，获得1保护点数。,合作合作+双箭头,5大洲:1盾,1声望,导游,C,[],[]
4,130,火龙研究专家,DRAGONOLOGIST,6,NaN,任何人+龙:2票,NaN,龙,NaN,NaN,任何动物园每打出1个龙类图标，你获得2票。For each dragon icon that...,若你的动物园内至少有3/6个龙类图标，获得1/2保护点数。Gain 1 conservati...,龙巢,3/6龙:1/2盾,-,北极熊馆(一模一样的框框尺寸),B,[AnimalTag.Dragon],[]


In [51]:
# 定义生成文本内容的函数
def generate_text(row):
    tags = '['+', '.join(row['tags']) + ']'
    requirements = '[' + ', '.join(row['requirements']) + ']'
    new_string = f"""
  {{
    id: '{row['编号']}',
    name: "{row['英文名']}",
    cnName: "{row['中文名']}",
    image: '/img/sponsors/{row['编号']}{row['中文名']}.png',
    strength: {row['强度需求']},
    rock: 0,
    water: 0,
    requirements: {requirements},
    tags: {tags},
    effects: [],
    reputation: 0,
    appeal: 0,
    conservationPoint: 0,
    source: CardSource.BASE,
  }},
"""
    return new_string

with open('sponsors.txt', 'w') as f:
    for index, row in df.iterrows():
        new_string = generate_text(row)
        f.write(new_string)


In [52]:
# 生成 CSS 图片

"""
.zoo-card.sponsor-card[data-id='S259_NativeLizards'] .ark-card-wrapper {
  background-image: url(/img/sponsors/S259_NativeLizards.jpg);
}
"""

# 定义生成文本内容的函数
def generate_text(row):
    id = row['编号']
    cnName = row['中文名']
    new_string = f"""
    .zoo-card.sponsor-card[data-id='{id}'] .ark-card-wrapper {{
  background-image: url(/img/sponsors/{id}{cnName}.png);

}}
"""
    return new_string

with open('sponsors_css.txt', 'w') as f:
    for index, row in df.iterrows():
        new_string = generate_text(row)
        f.write(new_string)

